In [3]:
import pandas as pd
from math import sqrt

#Read data

data = pd.read_csv('toy_dataset.csv')

user_data = {}
item_data = {}


for i in range(0, len(data)):
    user = data.loc[i, 'UserName']
    if user not in user_data:
        user_data[data.loc[i, 'UserName']] = {}
    user_data[user][data.loc[i, 'Product']] = data.loc[i, 'UserRating']

for i in range(0, len(data)):
    item = data.loc[i, 'Product']
    if item not in item_data:
        item_data[data.loc[i, 'Product']] = {}
    item_data[item][data.loc[i, 'UserName']] = data.loc[i, 'UserRating']
   



In [21]:
print user_data['plytle1']

{'Clinique Liquid Facial Soap': 9.0, 'Pevonia Botanica Sensitive Skin Mask': 10.0, 'Laboratories Luzern Organic Cosmeceuticals Serum Calm Absolut': 8.0, 'Bremenn Research Labs Hylexin Eye Cream': 7.0, 'Bioelements MAKEUP DISSOLVER': 5.0, "Kiehl's Epidermal Re-Texturizing Micro-Dermabrasion": 8.0, 'Coppertone ultraGuard Continuous Spray': 10.0, '3LAB Perfect Facial Wash': 5.0}


In [55]:
def pearson_similarity(person1, person2):
    
 
    num_keys = 0
    numerator = 0
    denominator = 0
    num_keys_person1 = 0
    num_keys_person2 = 0
    ratings_total_person1 = 0
    ratings_total_person2 = 0
    rating_diff_person1 = 0
    rating_diff_person2 = 0
    total_diff_person1_squared = 0
    total_diff_person2_squared = 0
    
    for key in person1:
        if key in person2:
            num_keys += 1
    if num_keys < 3:
        #print 'not enough overlap between users'
        return 0
    
    
    for key in person1:
        num_keys_person1 += 1
        ratings_total_person1 += person1[key]
    average_rating_person1 = ratings_total_person1 / num_keys_person1
    
    #print 'Person1 average rating: ', average_rating_person1
        
    for key in person2:
        num_keys_person2 += 1
        ratings_total_person2 += person2[key]
    average_rating_person2 = ratings_total_person2 / num_keys_person2
    
    #print 'Person2 average rating: ', average_rating_person2
    
    for key in person1:
        if key in person2:
            #print key, person1[key], person2[key]
            rating_diff_person1 = person1[key] - average_rating_person1
            rating_diff_person2 = person2[key] - average_rating_person2
            both_diff = rating_diff_person1 * rating_diff_person2
 
            numerator += both_diff
            
            total_diff_person1_squared += rating_diff_person1 ** 2
            total_diff_person2_squared += rating_diff_person2 ** 2
    
    denominator = sqrt(total_diff_person1_squared) * sqrt(total_diff_person2_squared)
    
    if num_keys == 0:
        return 0
    
    if denominator == 0:
        return 0
    else:
        return numerator/denominator
        

In [64]:
similarity_test = {}
similarity_test['A'] = {}
similarity_test['B'] = {}
similarity_test['A'][1] = 1
similarity_test['A'][2] = 2
similarity_test['A'][3] = 1
similarity_test['A'][4] = 7
similarity_test['A'][5] = 8
similarity_test['B'][1] = 7
similarity_test['B'][2] = 8
similarity_test['B'][3] = 9
similarity_test['B'][7] = 2
similarity_test['B'][8] = 3

print similarity_test
pearson_similarity(similarity_test['A'], similarity_test['B'])

{'A': {1: 1, 2: 2, 3: 1, 4: 7, 5: 8}, 'B': {8: 3, 1: 7, 2: 8, 3: 9, 7: 2}}


-0.9284766908852594

In [56]:
import sqlite3 as lite
import pandas as pd
db = lite.connect('capstone.db')
cur = db.cursor()
users = pd.read_sql_query('SELECT * FROM tblReview;', db)

In [6]:
users.head(5)

,gameID,movieID,tvShowID,author,publication,text,score,date,thumbsUp,thumbsDown,reviewType
0,1.0,NaN,NaN,,Areajugones,Ginger: Beyond the Crystal is a very good plat...,69,2016-11-05 00:00:00,0,0,c
1,1.0,NaN,NaN,,Hobby Consolas,"It\s not a bad game at all, but it has technic...",67,2016-11-02 00:00:00,0,0,c
2,1.0,NaN,NaN,,GameGrin,Ginger: Beyond the Crystal tries to revisit th...,60,2016-11-07 00:00:00,0,0,c
3,1.0,NaN,NaN,,GameSpew,It may not offer much of a challenge or much i...,60,2016-11-05 00:00:00,0,0,c
4,1.0,NaN,NaN,,Digitally Downloaded,"It may sound like I’m being harsh on Ginger, b...",50,2016-11-21 00:00:00,0,0,c


In [7]:
num_users = len(users)

item_reviews = {}

for i in range(0, num_users):

    game_id = users['gameID'][i]
    movie_id = users['movieID'][i]
    show_id = users['tvShowID'][i]
    score = users['score'][i]

    critic_review = users['reviewType'][i].encode('utf-8')

    if critic_review == 'c':
        user = users['publication'][i].encode('utf-8')
    else:
        user = users['author'][i].encode('utf-8')

    if pd.isnull(game_id) == False:
        if game_id not in item_reviews:
            item_reviews[game_id] = {}
        item_reviews[game_id][user] = score
    
    if pd.isnull(movie_id) == False:
        if movie_id not in item_reviews:
            item_reviews[movie_id] = {}
        item_reviews[movie_id][user] = score
    
    if pd.isnull(show_id) == False:
        if show_id not in item_reviews:
            item_reviews[show_id] = {}
        item_reviews[show_id][user] = score

In [8]:
for key in item_reviews:
    try:
        del item_reviews[key]['']
    except KeyError:
        pass

In [9]:
itemlist = []
for key in item_reviews:
    itemlist.append(key)

In [10]:
len(itemlist)

20140

In [11]:
review_sims = pd.DataFrame(index = itemlist, columns = itemlist)

In [48]:
review_sims.head()

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,20160.0,20161.0,20162.0,20163.0,20164.0,20165.0,20166.0,20167.0,20168.0,20169.0
1.0,1,0.492379,0.989582,0.473033,0.223733,0.864406,0.523792,0.320626,0.155217,0.705438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,0.492379,1,0.166898,0.368561,0.195442,0.648521,0.583082,0.50858,0.120628,0.357676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,0.989582,0.166898,1,0.299662,0.0215428,0.285365,0.425651,0.175776,0.195412,0.34595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,0.473033,0.368561,0.299662,1,0.157985,0.566472,0.466845,0.666028,0.00872126,0.668568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,0.223733,0.195442,0.0215428,0.157985,1,0.3166,0.336052,0.326698,0.0965787,0.140852,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:

#for i in range(0, len(review_sims.index)):
#    for j in range(0, len(review_sims.index)):
for i in range(0, 10):
    for j in range(0, 10):
        item1 = review_sims.index[i]
        item2 = review_sims.columns[j]

        if i == j:
            review_sims.iloc[i, j] = 1
        
        else:
            review_sims.iloc[i, j] = pearson_similarity(item_reviews[item1], item_reviews[item2])





In [50]:
review_sims.head(10)

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,20160.0,20161.0,20162.0,20163.0,20164.0,20165.0,20166.0,20167.0,20168.0,20169.0
1.0,1,0.492379,0.989582,0.473033,0.223733,0.864406,0.523792,0.320626,0.155217,0.705438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,0.492379,1,0.166898,0.368561,0.195442,0.648521,0.583082,0.50858,0.120628,0.357676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,0.989582,0.166898,1,0.299662,0.0215428,0.285365,0.425651,0.175776,0.195412,0.34595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,0.473033,0.368561,0.299662,1,0.157985,0.566472,0.466845,0.666028,0.00872126,0.668568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,0.223733,0.195442,0.0215428,0.157985,1,0.3166,0.336052,0.326698,0.0965787,0.140852,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6.0,0.864406,0.648521,0.285365,0.566472,0.3166,1,0.603133,0.733373,0.351049,0.735194,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7.0,0.523792,0.583082,0.425651,0.466845,0.336052,0.603133,1,0.446067,0.310692,0.587536,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.0,0.320626,0.50858,0.175776,0.666028,0.326698,0.733373,0.446067,1,0.271044,0.67083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9.0,0.155217,0.120628,0.195412,0.00872126,0.0965787,0.351049,0.310692,0.271044,1,0.621582,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.0,0.705438,0.357676,0.34595,0.668568,0.140852,0.735194,0.587536,0.67083,0.621582,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
pearson_similarity(item_reviews[1], item_reviews[2])

0.49237941116434775